In [1]:
import pandas as pd 
import yaml
from pathlib import Path
from itertools import product
from sklearn.model_selection import KFold, train_test_split
from ultralytics import YOLO
import os

In [2]:
ROOT = Path(r"/Users/pepefv97/Downloads/DATOS_PLACAS/train")  # raíz original
IMG_DIR = ROOT / "images"
LBL_DIR = ROOT / "labels"

In [34]:
imgs = sorted(IMG_DIR.rglob("*.jpg"))
df   = pd.DataFrame({"img": imgs})
train_df = df.sample(frac=0.10, random_state=123).reset_index(drop=True)


In [35]:
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=123)


In [36]:
OUT_ROOT = Path("folds")                     # carpeta raíz donde viven todos los folds
OUT_ROOT.mkdir(exist_ok=True)

fold_paths = []

for fold_i, (tr, va) in enumerate(kf.split(train_df)):
    tr_imgs = train_df.iloc[tr]["img"].tolist()
    va_imgs = train_df.iloc[va]["img"].tolist()

    # --- 1. Crear la estructura de carpetas -------------------------
    fold_dir = OUT_ROOT / f"fold{fold_i}"
    for split in ["train", "val"]:
        for sub in ["images", "labels"]:
            (fold_dir / split / sub).mkdir(parents=True, exist_ok=True)

    # --- 2. Hard‑links para imágenes y labels -----------------------
    def link(img_path, dst_img_dir):
        src_img = Path(img_path)
        src_lbl = LBL_DIR / src_img.with_suffix(".txt").name
        dst_img = dst_img_dir / src_img.name
        dst_lbl = dst_img_dir.parent / "labels" / src_lbl.name
        os.link(src_img, dst_img)   # crea enlace duro a la imagen
        os.link(src_lbl, dst_lbl)   # crea enlace duro al .txt

    for p in tr_imgs:
        link(p, fold_dir / "train" / "images")
    for p in va_imgs:
        link(p, fold_dir / "val" / "images")

    # --- 3. YAML: usa rutas directas de este fold (¡sin duplicar!) --
    yaml.dump(
        {
            "train": str((fold_dir / "train" / "images").resolve()),
            "val":   str((fold_dir / "val"   / "images").resolve()),
            "nc": 1,
            "names": ["PLACA_AUTOMOVIL"]
        },
        open(fold_dir / "data.yaml", "w")
    )

    fold_paths.append(fold_dir / "data.yaml")   # guardar ruta para entrenos posteriores


In [37]:
param_grid = {
    "optimizer": ["SGD", "AdamW"],             # mejores en detección :contentReference[oaicite:3]{index=3}
    "lr0": [0.001, 0.005],
    "weight_decay": [1e-4, 5e-4],
    "mosaic": [0.5, 1.0]
}
grid = [dict(zip(param_grid, v)) for v in product(*param_grid.values())]

params_train = dict(epochs=3, batch=32, imgsz=640,
              device="mps", freeze=10, plots=True, verbose=True)


In [38]:
for cfg in grid:
    for fold_yaml in fold_paths:
        name = (
            f"{Path(fold_yaml).parent.name}"
            f"_opt-{cfg['optimizer']}"
            f"_lr{cfg['lr0']}"
            f"_wd{cfg['weight_decay']}"
            f"_mos{cfg['mosaic']}"
        )
        YOLO("yolo11n.pt").train(
            data=str(fold_yaml),
            project="cross_validation_hpt",
            name=name,
            **cfg,
            **params_train
        )

100%|██████████████████████████████████████| 5.35M/5.35M [00:02<00:00, 2.56MB/s]


New https://pypi.org/project/ultralytics/8.3.170 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=folds/fold0/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=fold0_opt-SGD_lr0.001_wd0.0001_mos0.5, nbs=64, nms=False, opset=None, optimiz

train: Scanning /Users/pepefv97/Downloads/PROYECTO_5/HYPERPARAMETER_TUNING/folds

train: New cache created: /Users/pepefv97/Downloads/PROYECTO_5/HYPERPARAMETER_TUNING/folds/fold0/train/labels.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 243.8±75.8 MB/s, size: 37.7 KB)


...

        3/3      5.39G      1.379     0.9566      1.386         49        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        423        445      0.776      0.843      0.814      0.479



3 epochs completed in 0.042 hours.
Optimizer stripped from cross_validation_hpt/fold4_opt-AdamW_lr0.005_wd0.0005_mos1.0/weights/last.pt, 5.5MB
Optimizer stripped from cross_validation_hpt/fold4_opt-AdamW_lr0.005_wd0.0005_mos1.0/weights/best.pt, 5.5MB

Validating cross_validation_hpt/fold4_opt-AdamW_lr0.005_wd0.0005_mos1.0/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        423        445      0.776      0.843      0.814      0.479
Speed: 1.9ms preprocess, 3.5ms inference, 0.0ms loss, 7.9ms postprocess per image
Results saved to cross_validation_hpt/fold4_opt-AdamW_lr0.005_wd0.0005_mos1.0


In [40]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("cross_validation_hpt")   # carpeta raíz de tus runs
pattern   = re.compile(
    r"fold\d+_"            # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)  # ← extrae hiperparámetros del nombre de carpeta

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue  # ignora carpetas que no sigan el patrón
    hp = m.groupdict()                 # dict con opt, lr, wd, mos
    df = pd.read_csv(csv)
    best_map50 = df["metrics/mAP50(B)"].max()     # mejor época
    fold       = run_dir.name.split("_")[0]     # fold0, fold1…
    records.append({**hp, "fold": fold, "mAP50": best_map50})

# -------------------------------------------------------------
# PROMEDIAR Y CALCULAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]
summary = (
    df.groupby(group_cols)["mAP50"]
      .agg(["mean", "std", "count"])
      .reset_index()
      .sort_values("mean", ascending=False)
)
print(summary.head(5))


      opt     lr      wd  mos      mean       std  count
1   AdamW  0.001  0.0001  1.0  0.950604  0.015525      5
3   AdamW  0.001  0.0005  1.0  0.950512  0.016957      5
14    SGD  0.005  0.0005  0.5  0.949240  0.019162      5
2   AdamW  0.001  0.0005  0.5  0.948540  0.016219      5
12    SGD  0.005  0.0001  0.5  0.945874  0.017130      5
15    SGD  0.005  0.0005  1.0  0.942148  0.018745      5
13    SGD  0.005  0.0001  1.0  0.940110  0.025150      5
10    SGD  0.001  0.0005  0.5  0.922364  0.024646      5
5   AdamW  0.005  0.0001  1.0  0.914466  0.026179      5
6   AdamW  0.005  0.0005  0.5  0.897924  0.029198      5


In [3]:
from ultralytics import YOLO

path_yml = r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml'


model = YOLO("yolo11n.pt")      
model.train(
    data=path_yml,      
    epochs=10,                
    batch=32,
    imgsz=640,
    device="mps",
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0001,
    mosaic=1.0,                
    freeze=10,                  
    plots=True,
    project="ENTRENAMIENTO_YOLOV11",
    name="YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0",
    verbose=True
)

New https://pypi.org/project/ultralytics/8.3.170 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0, nbs=64, nms=

train: Scanning /Users/pepefv97/Downloads/DATOS_PLACAS/train/labels.cache... 211

val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 165.8±65.5 MB/s, size: 35.9 KB)



val: Scanning /Users/pepefv97/Downloads/DATOS_PLACAS/valid/labels.cache... 2046 


Plotting labels to ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2046       2132      0.976      0.963      0.986      0.704



10 epochs completed in 3.032 hours.
Optimizer stripped from ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/last.pt, 5.5MB
Optimizer stripped from ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/best.pt, 5.5MB

Validating ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       2046       2132      0.976      0.963      0.986      0.704
Speed: 0.9ms preprocess, 11.9ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x3cd50ba40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [2]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("cross_validation_hpt")
pattern   = re.compile(
    r"fold\d+_"                # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue                                           # ignora carpetas mal nombradas
    hp = m.groupdict()                                     # hiperparámetros
    df = pd.read_csv(csv)

    # --- localizar la FILA con el mejor mAP50 ---
    best_row        = df.loc[df["metrics/mAP50(B)"].idxmax()]
    best_map50      = best_row["metrics/mAP50(B)"]
    best_map5095    = best_row["metrics/mAP50-95(B)"]      # <- misma época

    records.append({
        **hp,
        "fold"     : run_dir.name.split("_")[0],           # fold0, fold1…
        "mAP50"    : best_map50,
        "mAP50_95" : best_map5095
    })

# -------------------------------------------------------------
# PROMEDIAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]

summary = (
    df.groupby(group_cols)
      .agg(
          mean    = ("mAP50",    "mean"),   # igual que antes
          std     = ("mAP50",    "std"),
          count   = ("fold",     "count"),
          mean95  = ("mAP50_95", "mean"),   # nuevas columnas
          std95   = ("mAP50_95", "std")
      )
      .reset_index()
      .sort_values("mean", ascending=False)  # mismo criterio de orden
)

print(summary.head(5))


      opt     lr      wd  mos      mean       std  count    mean95     std95
1   AdamW  0.001  0.0001  1.0  0.950604  0.015525      5  0.599796  0.028085
3   AdamW  0.001  0.0005  1.0  0.950512  0.016957      5  0.602044  0.014005
14    SGD  0.005  0.0005  0.5  0.949240  0.019162      5  0.601894  0.013597
2   AdamW  0.001  0.0005  0.5  0.948540  0.016219      5  0.607010  0.012551
12    SGD  0.005  0.0001  0.5  0.945874  0.017130      5  0.606122  0.011670


In [4]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("yolov11_cross_validation_hpt")
pattern   = re.compile(
    r"fold\d+_"                # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue                                           # ignora carpetas mal nombradas
    hp = m.groupdict()                                     # hiperparámetros
    df = pd.read_csv(csv)

    # --- localizar la FILA con el mejor mAP50 ---
    best_row        = df.loc[df["metrics/mAP50(B)"].idxmax()]
    best_map50      = best_row["metrics/mAP50(B)"]
    best_map5095    = best_row["metrics/mAP50-95(B)"]      # <- misma época

    records.append({
        **hp,
        "fold"     : run_dir.name.split("_")[0],           # fold0, fold1…
        "mAP50"    : best_map50,
        "mAP50_95" : best_map5095
    })

# -------------------------------------------------------------
# PROMEDIAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]

summary = (
    df.groupby(group_cols)
      .agg(
          mean    = ("mAP50",    "mean"),   # igual que antes
          std     = ("mAP50",    "std"),
          count   = ("fold",     "count"),
          mean95  = ("mAP50_95", "mean"),   # nuevas columnas
          std95   = ("mAP50_95", "std")
      )
      .reset_index()
      .sort_values("mean", ascending=False)  # mismo criterio de orden
)

print(summary.head(5))


     opt      lr      wd  mos      mean       std  count    mean95     std95
2  AdamW  0.0005  0.0005    0  0.968620  0.011585      5  0.629148  0.007569
3  AdamW  0.0005  0.0005  0.5  0.966394  0.010837      5  0.623744  0.025070
0  AdamW  0.0005  0.0001    0  0.964902  0.014017      5  0.626980  0.011902
1  AdamW  0.0005  0.0001  0.5  0.964328  0.011176      5  0.622514  0.018757
7  AdamW   0.001  0.0005  0.5  0.958630  0.012866      5  0.598000  0.019092


In [5]:
# -------------------------------------------------------------
# LEER TODOS LOS metrics.csv Y AGREGAR POR COMBINACIÓN
# -------------------------------------------------------------
import glob, re, pandas as pd
from pathlib import Path

ROOT_RUNS = Path("yolov10_cross_validation_hpt")
pattern   = re.compile(
    r"fold\d+_"                # foldX_
    r"opt-(?P<opt>\w+)_"
    r"lr(?P<lr>[\d.]+)_"
    r"wd(?P<wd>[\d.]+)_"
    r"mos(?P<mos>[\d.]+)"
)

records = []

for csv in ROOT_RUNS.rglob("results.csv"):
    run_dir = csv.parent
    m = pattern.search(run_dir.name)
    if not m:
        continue                                           # ignora carpetas mal nombradas
    hp = m.groupdict()                                     # hiperparámetros
    df = pd.read_csv(csv)

    # --- localizar la FILA con el mejor mAP50 ---
    best_row        = df.loc[df["metrics/mAP50(B)"].idxmax()]
    best_map50      = best_row["metrics/mAP50(B)"]
    best_map5095    = best_row["metrics/mAP50-95(B)"]      # <- misma época

    records.append({
        **hp,
        "fold"     : run_dir.name.split("_")[0],           # fold0, fold1…
        "mAP50"    : best_map50,
        "mAP50_95" : best_map5095
    })

# -------------------------------------------------------------
# PROMEDIAR (media ± std) POR COMBINACIÓN
# -------------------------------------------------------------
df = pd.DataFrame(records)
group_cols = ["opt", "lr", "wd", "mos"]

summary = (
    df.groupby(group_cols)
      .agg(
          mean    = ("mAP50",    "mean"),   # igual que antes
          std     = ("mAP50",    "std"),
          count   = ("fold",     "count"),
          mean95  = ("mAP50_95", "mean"),   # nuevas columnas
          std95   = ("mAP50_95", "std")
      )
      .reset_index()
      .sort_values("mean", ascending=False)  # mismo criterio de orden
)

print(summary.head(5))


     opt      lr      wd  mos      mean       std  count    mean95     std95
2  AdamW  0.0005  0.0005    0  0.944558  0.012567      5  0.619040  0.018822
0  AdamW  0.0005  0.0001    0  0.944364  0.022141      5  0.609828  0.020138
1  AdamW  0.0005  0.0001  0.5  0.937448  0.021759      5  0.597804  0.013135
3  AdamW  0.0005  0.0005  0.5  0.934474  0.021331      5  0.586242  0.028405
5  AdamW   0.001  0.0001  0.5  0.929850  0.024330      5  0.575876  0.024535
